In [1]:
import csv
import requests
import pandas as pd
import numpy as np
import csv
import codecs
import urllib.request
import urllib.error
import sys
import glob
import urllib.parse
import urllib.request
import json
import time
import datetime
from datetime import datetime, timedelta

In [3]:
regions = pd.read_csv('regions.csv')

In [5]:
alarms = pd.read_csv('alarms.csv', sep = ';')

In [6]:
weather = pd.read_csv('all_weather_by_hour.csv')


In [ ]:
def isNaN(num):
    return num != num

In [ ]:
df_isw = pd.read_csv(f"isw_data.csv", sep=",")
df_isw.head(5)

In [ ]:
df_weather = pd.read_csv(f"all_weather_by_hour_v2.csv")
df_weather["day_datetime"] = pd.to_datetime(df_weather["day_datetime"])

In [ ]:
df_weather.shape


In [ ]:
df_weather.head(15)

In [ ]:
weather_exclude = [
"day_feelslikemax",
"day_feelslikemin",
"day_sunriseEpoch",
"day_sunsetEpoch",
"day_description",
"city_latitude",
"city_longitude",
"city_address",
"city_timezone",
"city_tzoffset",
"day_feelslike",
"day_precipprob",
"day_snow",
"day_snowdepth",
"day_windgust",
"day_windspeed",
"day_winddir",
"day_pressure",
"day_cloudcover",
"day_visibility",
"day_severerisk",
"day_conditions",
"day_icon",
"day_source",
"day_preciptype",
"day_stations",
"hour_icon",
"hour_source",
"hour_stations",
"hour_feelslike"
]

In [ ]:
df_weather_v2 = df_weather.drop(weather_exclude, axis=1)

In [ ]:
df_weather_v2.head(5)

In [ ]:
df_weather_v2["city"] = df_weather_v2["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
df_weather_v2["city"] = df_weather_v2["city"].replace('Хмельницька область', "Хмельницький")

In [ ]:
df_weather_v2.shape

In [ ]:
df_regions = pd.read_csv(f"regions.csv")
df_weather_reg = pd.merge(df_weather_v2, df_regions, left_on="city",right_on="center_city_ua")

In [ ]:
df_events = pd.read_csv(f"alarms.csv", sep=";")
df_events_v2 = df_events.drop(["id","region_id"],axis=1)
df_events_v2.head(5)

In [ ]:
df_events_v2["start_time"] = pd.to_datetime(df_events_v2["start"])
df_events_v2["end_time"] = pd.to_datetime(df_events_v2["end"])

df_events_v2["start_hour"] = df_events_v2['start_time'].dt.floor('H')
df_events_v2["end_hour"] = df_events_v2['end_time'].dt.ceil('H')

df_events_v2["start_hour"] = df_events_v2.apply(lambda x: x["start_hour"] if not isNaN(x["start_hour"]) else x["event_hour"] , axis=1)
df_events_v2["end_hour"] = df_events_v2.apply(lambda x: x["end_hour"] if not isNaN(x["end_hour"]) else x["event_hour"] , axis=1)

df_events_v2["day_date"] = df_events_v2["start_time"].dt.date

df_events_v2["start_hour_datetimeEpoch"] = df_events_v2['start_hour'].apply(lambda x: int(x.strftime('%s'))  if not isNaN(x) else None)
df_events_v2["end_hour_datetimeEpoch"] = df_events_v2['end_hour'].apply(lambda x: int(x.strftime('%s'))  if not isNaN(x) else None)

df_events_v2.head(10)

In [ ]:
events_dict = df_events_v2.to_dict('records')
events_by_hour = []

In [ ]:
for event in events_dict:
    for d in pd.date_range(start=event["start_hour"], end=event["end_hour"], freq='1H'):
        et = event.copy()
        et["hour_level_event_time"] = d
        events_by_hour.append(et)

In [ ]:
df_events_v3 = pd.DataFrame.from_dict(events_by_hour)

df_events_v3["hour_level_event_datetimeEpocdh"] = df_events_v3["hour_level_event_time"].apply(lambda x: int(x.strftime('%s'))  if not isNaN(x) else None)

df_events_v3.shape

In [ ]:
df_events_v3.head(15)

In [ ]:
df_weather_v4 = df_weather_reg.merge(df_events_v4, 
                                     how="left", 
                                     left_on=["region_alt","hour_datetimeEpoch"],
                                     right_on=["event_region_title","event_hour_level_event_datetimeEpocdh"])

df_weather_v4.head(10)

In [ ]:
df_weather_v4.to_csv(f"all_data_preprocessed.csv", sep=";", index=False)

In [ ]:
df = pd.read_csv("all_data_preprocessed.csv", sep=';')
df.head()
df['day_datetime'] = pd.to_datetime(df_isw_text["day_datetime"])
df = df.drop(['Unnamed: 0'], axis=1)
df.dtypes
df.to_csv("test.csv", sep=';')
df['alarm_info'] = np.where(isNaN(df['event_start']),0,1)
df.head()
df.to_csv("test.csv", sep=';', index=False)

In [ ]:
df1 = pd.read_csv('vectorized_texts.csv', sep=',')
df_isw = pd.read_csv('isw_data.csv', sep=',')

with open('cols_vect.txt', 'r') as f:
    columns = [col.strip() for col in f.read().strip().split(',')]

for col in df1.columns:
    if col not in columns:
        df1.drop(col, axis=1, inplace=True)

df1['day_datetime'] = pd.to_datetime(df_isw['Date'])
df1['day_datetime'] = df1['day_datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
df2 = pd.read_csv('test.csv', sep=';', low_memory=False)
df2['alarm_info'] = df2['alarm']

# merge the two datasets on the common column
merged_df = pd.merge(df2, df1, on='day_datetime', how='inner')
merged_df.drop('Unnamed: 0', axis=1, inplace=True)

# delete all NaN columns
nan_cols = merged_df.isna().any()
cols_with_nan = merged_df.columns[nan_cols]
merged_df = merged_df.drop(columns=cols_with_nan)

# delete all object columns
merged_df = merged_df.select_dtypes(exclude=['object'])

merged_df = merged_df.drop(
    ['day_datetimeEpoch', 'day_precipcover', 'day_solarradiation', 'day_solarenergy', 'hour_datetimeEpoch',
     'hour_precip', 'hour_precipprob', 'hour_snowdepth', 'hour_windspeed', 'hour_visibility', 'hour_cloudcover',
     'hour_severerisk', 'alarm_x', 'alarm_y'], axis=1)

# write the merged dataset to a new CSV file
merged_df.to_csv('merged_dataset.csv', index=False)
print(merged_df.dtypes)
